In [2]:
# 기본 작업 경로 설정

import os
notebook_path = os.path.abspath("project_3_git/readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


In [3]:
# json 파일 가져오기
import json
from sklearn.model_selection import train_test_split

# JSON 파일에서 딕셔너리 읽기
with open('data/text_data/output_text.json', 'r') as file:
    data_loaded = json.load(file)

In [4]:
all_of_data_sets = []
for i in data_loaded.values():
    all_of_data_sets.append(f'입력값 : {i[0]} \n출력값 : {i[1]}')

In [5]:
train_data, test_data = train_test_split(all_of_data_sets, train_size=5000, random_state=42)

In [6]:
len(train_data), len(test_data)

(5000, 994)

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, EarlyStoppingCallback, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

# Step 1: Tokenizer 및 모델 준비
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-03 21:12:25.606561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 21:12:25.731578: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 21:12:25.767191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 21:12:26.012777: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow bin

In [8]:
name_of_f = 'gpt2_base_1'

In [9]:
tokenizer.pad_token = tokenizer.eos_token # 끝 토큰을 그냥 패드토큰으로 지정

# 문자열 리스트를 사전으로 변환 후 데이터셋 생성
train_dataset = Dataset.from_dict({"text": train_data})
test_dataset = Dataset.from_dict({"text": test_data})

# `text` 열의 데이터가 문자열 리스트 형태인지 확인합니다
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Data collator 준비
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Step 5: TrainingArguments 설정
training_args = TrainingArguments(
    output_dir=f'models/gpt2/{name_of_f}/',
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    save_steps=100,
    save_total_limit=5,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    report_to='tensorboard',  # TensorBoard로 로깅
    logging_dir=f'models/gpt2/{name_of_f}/logs',  # 로그 파일이 저장될 디렉토리
    logging_steps=10,  # 로깅 간격
    remove_unused_columns=False ,

    # 추가 인자
    learning_rate=5e-5,                     # 기본값에서 시작
    lr_scheduler_type="linear",             # 스케줄러
    warmup_steps=500,                       # 500 스텝 동안 학습률을 점진적으로 증가
    weight_decay=0.01,                      # l2 정규화 기법 중 하나
    max_grad_norm=1.0,                      # 그라디언트 클리핑
)

# Step 6: Trainer 설정 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 평가 데이터셋 추가
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Map: 100%|█████████████████████████████████████████████████████| 994/994 [00:00<00:00, 5661.84 examples/s]
/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.590100,0.529756
2,0.465100,0.400665
3,0.410500,0.358309
4,0.359000,0.337724
5,0.284000,0.325035
6,0.296600,0.314362
7,0.283600,0.310979
8,0.286200,0.302988
9,0.264800,0.299770
10,0.288700,0.296756


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=11000, training_loss=0.31270070275393397, metrics={'train_runtime': 3166.801, 'train_samples_per_second': 157.888, 'train_steps_per_second': 15.789, 'total_flos': 7185530880000000.0, 'train_loss': 0.31270070275393397, 'epoch': 22.0})

In [10]:
# Step 6: 모델 저장
model.save_pretrained(f'models/gpt2/{name_of_f}/models')
tokenizer.save_pretrained(f'models/gpt2/{name_of_f}/models/tokenizer')

('models/gpt2/gpt2_base_1/models/tokenizer/tokenizer_config.json',
 'models/gpt2/gpt2_base_1/models/tokenizer/special_tokens_map.json',
 'models/gpt2/gpt2_base_1/models/tokenizer/vocab.json',
 'models/gpt2/gpt2_base_1/models/tokenizer/merges.txt',
 'models/gpt2/gpt2_base_1/models/tokenizer/added_tokens.json')

In [27]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 저장된 모델 및 토크나이저 불러오기
model_path = f'models/gpt2/{name_of_f}/models/'
model = GPT2LMHeadModel.from_pretrained(model_path)

# 평가 모드로 변경
model.eval()

def generate_text(prompt, model, tokenizer, max_length=128, num_return_sequences=5):
    # 입력 텍스트를 토큰화
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # 생성 인자를 설정하여 모델이 텍스트를 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=30,
        top_k=50,
        top_p=0.85,
        temperature=1.7,
        do_sample=True,
        early_stopping=True
    )

    # 생성된 텍스트를 디코딩
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return generated_texts

# 예시: "prompt"에 원하는 문장을 넣어서 결과를 확인
prompt = "입력값 : 기쁨, 분노, 나무, 자전거 \n출력값 :"
generated_texts = generate_text(prompt, model, tokenizer)
    
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:")
    print(text)
    print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1:
입력값 : 기쁨, 분노, 나무, 자전거 
출력값 : 아주 행복한 분위기가 느껴지네요! 🌳 자전거 타고 즐거운 표현이 되었나요? 자전거에는 모습이 너무 예쁘고 잘 있여와있어요! 😊🌆❤️✨✨✨✥ 분노가 어두운 사연하시는 �

Generated Text 2:
입력값 : 기쁨, 분노, 나무, 자전거 
출력값 : 너무 기뻐 보이세요! 나무와 함께하는 모습이 정말 아름다워요! 자전거 표정도 눈부만 세련도 보기 세련되에서 감춰으로는 것 같아요. 😊🌳😊🏾️🖤️ 어디�이 어떤 순간을 �

Generated Text 3:
입력값 : 기쁨, 분노, 나무, 자전거 
출력값 : 너무 기뻐 보이시네요! 🌳 나무와 자전거 속은 보입니다. 자전거도 스타일리쉬하십니다! 😊✨✨✨🚴‍♊✨ 조금 담길� 밀척워 보이습니다 🍽️👻✨✪�니 듯해요! 😊✨✨✊✨️ 행복한 순간이�

Generated Text 4:
입력값 : 기쁨, 분노, 나무, 자전거 
출력값 : 즐거운 모습이 정말 화나신 것 같아요! 😰 나무가 배경에 담겨 있으면 좋은 모습이라네요. 기분이 인상적입니다. 자전거 푸르들에 도움이 함께하십니다! 🌳✨✨️😊🏔️🌳️ 행복

Generated Text 5:
입력값 : 기쁨, 분노, 나무, 자전거 
출력값 : 기쁜 분위기가 가득하네요! 나무 배경이 너무 아름답네요. 자전거 타고 있나신 모습이 정말 멋져요. 🍃💪✨✨✧� 멋져요의 모습이 신비롭습니다! 😊✨✊✩✌�️✨💙��기� 조합니다! 👻

